In [1]:
import os
from typing import List, Any
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import (
    TextLoader, PyPDFLoader, CSVLoader, Docx2txtLoader,
    UnstructuredPowerPointLoader, UnstructuredHTMLLoader,
    UnstructuredMarkdownLoader, UnstructuredExcelLoader
)
from langchain_community.llms import Ollama


In [2]:
# 전역 설정 변수
CHUNK_SIZE = 500  # 텍스트 분할 시 각 청크의 크기
CHUNK_OVERLAP = 50  # 텍스트 청크 간 중복되는 부분의 크기
TEMPERATURE = 0.1  # 언어 모델의 온도 (낮을수록 더 일관된 출력)
MAX_TOKENS = 512  # 생성할 최대 토큰 수
NUM_RETRIEVED_DOCUMENTS = 3  # 검색할 관련 문서의 수
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"  # 사용할 임베딩 모델
OLLAMA_MODEL = "llama3.1" # 사용할 Ollama 모델

In [3]:
class DataLoader:
    """다양한 형식의 문서를 로드하는 클래스"""

    def __init__(self):
        # 지원하는 파일 확장자와 해당 로더를 매핑
        self.loaders = {
            '.txt': TextLoader,
            '.pdf': PyPDFLoader,
            '.csv': CSVLoader,
            '.xlsx': UnstructuredExcelLoader,
            '.xls': UnstructuredExcelLoader,
            '.docx': Docx2txtLoader,
            '.pptx': UnstructuredPowerPointLoader,
            '.html': UnstructuredHTMLLoader,
            '.md': UnstructuredMarkdownLoader
        }

    def load(self, path: str) -> List[Any]:
        """
        주어진 경로의 파일 또는 디렉토리에서 문서를 로드
        
        :param path: 로드할 파일 또는 디렉토리 경로
        :return: 로드된 문서 리스트
        """
        path = os.path.normpath(os.path.expanduser(path))
        if not os.path.exists(path):
            raise ValueError(f"경로가 존재하지 않습니다: {path}")
        
        if os.path.isfile(path):
            return self._load_file(path)
        elif os.path.isdir(path):
            return self._load_directory(path)
        else:
            raise ValueError(f"지원하지 않는 경로입니다: {path}")

    def _load_file(self, file_path: str) -> List[Any]:
        """단일 파일을 로드"""
        _, ext = os.path.splitext(file_path.lower())
        if ext not in self.loaders:
            raise ValueError(f"지원하지 않는 파일 형식입니다: {ext}")
        loader = self.loaders[ext](file_path)
        return loader.load()

    def _load_directory(self, dir_path: str) -> List[Any]:
        """디렉토리 내의 모든 지원되는 파일을 로드"""
        documents = []
        for root, _, files in os.walk(dir_path):
            for file in files:
                file_path = os.path.join(root, file)
                try:
                    documents.extend(self._load_file(file_path))
                except ValueError as e:
                    print(f"경고: {file_path} 로딩 중 오류 발생 - {str(e)}")
        return documents


In [4]:
class ChatBot:
    """대화형 AI 챗봇 클래스"""

    def __init__(self, data_path: str):
        """
        ChatBot 초기화
        
        :param data_path: 학습 데이터 파일 또는 디렉토리 경로
        """
        self.data_path = data_path
        self.data_loader = DataLoader()
        self.embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)
        self.llm = Ollama(model=OLLAMA_MODEL, temperature=TEMPERATURE)
        self.memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        self.qa_chain = None

    def setup(self):
        """챗봇 시스템 초기화 및 설정"""
        print("시스템을 초기화하는 중입니다...")
        # 문서 로드
        documents = self.data_loader.load(self.data_path)
        # 텍스트 분할
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
        chunks = text_splitter.split_documents(documents)
        # 벡터 저장소 생성
        vectorstore = FAISS.from_documents(chunks, self.embeddings)

        # 프롬프트 템플릿 정의
        prompt_template = """
당신은 다양한 주제에 대해 지식을 갖춘 AI 어시스턴트입니다. 사용자의 질문에 대해 친절하고 정확하게 답변해 주세요. 다음 지침을 따라주세요:

1. 항상 정중하고 전문적인 태도를 유지하세요.
2. 질문에 대해 명확하고 간결하게 답변하되, 필요한 경우 자세한 설명을 제공하세요.
3. 사실에 기반한 정보를 제공하고, 추측이나 불확실한 정보는 피하세요.
4. 질문의 맥락을 고려하여 적절한 답변을 제공하세요.
5. 윤리적이고 건설적인 방식으로 대화에 임하세요.
6. 질문이 불명확하거나 추가 정보가 필요한 경우, 명확화를 요청하세요.
7. 복잡한 개념은 쉽게 이해할 수 있도록 설명하세요.
8. 사용자의 지식 수준에 맞춰 답변의 깊이를 조절하세요.
9. 다양한 관점을 제시할 수 있는 주제에 대해서는 균형 잡힌 시각을 제공하세요.
10. 귀하의 한계를 인정하고, 모르는 것에 대해서는 솔직히 모른다고 말씀하세요.

대화를 통해 사용자의 요구사항을 파악하고, 최선의 도움을 제공하기 위해 노력하세요. 필요한 경우 예시를 들거나 단계별 설명을 제공할 수 있습니다.

참고 데이터: {context}

사용자: {question}

AI 어시스턴트:
"""
        
        qa_prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

        # ConversationalRetrievalChain 설정
        self.qa_chain = ConversationalRetrievalChain.from_llm(
            llm=self.llm,
            retriever=vectorstore.as_retriever(search_kwargs={"k": NUM_RETRIEVED_DOCUMENTS}),
            memory=self.memory,
            combine_docs_chain_kwargs={"prompt": qa_prompt}
        )
        print("초기화가 완료되었습니다.")

    def ask(self, query: str) -> str:
        """
        사용자의 질문에 대한 답변 생성
        
        :param query: 사용자의 질문
        :return: AI의 답변
        """
        if not self.qa_chain:
            self.setup()
        try:
            response = self.qa_chain({"question": query})
            return response['answer']
        except Exception as e:
            return f"죄송합니다. 질문을 처리하는 중에 오류가 발생했습니다: {str(e)}"

    def chat(self):
        """대화형 인터페이스 실행"""
        print("챗봇이 준비되었습니다. '종료'를 입력하면 대화를 마칩니다.")
        while True:
            query = input("질문을 입력하세요: ")
            if query.lower() == '종료':
                break
            answer = self.ask(query)
            print("\n답변:", answer, "\n")
        print("대화를 종료합니다.")

In [5]:
def main():
    """메인 함수: 챗봇 실행"""
    # data_path = input("데이터 파일 또는 폴더 경로를 입력하세요: ")
    data_path = "/Volumes/JD's box2/music_produce/음악공부 자료/침실프로듀서의+EQ챠트.pdf"
    chatbot = ChatBot(data_path=data_path)
    chatbot.chat()

if __name__ == "__main__":
    main()

/Users/joyong-eun/Desktop/Dev/Langchain/bible/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/joyong-eun/Desktop/Dev/Langchain/bible/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


챗봇이 준비되었습니다. '종료'를 입력하면 대화를 마칩니다.
시스템을 초기화하는 중입니다...
초기화가 완료되었습니다.


/Users/joyong-eun/Desktop/Dev/Langchain/bible/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(



답변: 트랭크 음악의 킥 사운드에 대한 EQ 설정은 중요합니다. 일반적으로, 킥 사운드는 저음역대(주로 60 Hz ~ 100 Hz)에서 강한 에너지를 가지고 있습니다. 이 영역을 강화하여 킥 사운드가 더 선명하고 강력하게 들리도록 할 수 있습니다.

킥 EQ 설정의 일반적인 순서는 다음과 같습니다:

1. **Low-Sub (초저음)**: 20 Hz ~ 60 Hz 영역에서 약간의 에너지를 추가하여 킥 사운드의 깊이를 增强합니다.
2. **Sub Frequency (킥, 베이스 음역대)**: 60 Hz ~ 100 Hz 영역에서 강한 에너지를 추가하여 킥 사운드가 더 선명하고 강력하게 들리도록 합니다.
3. **Thickness (악기들의 저음역)**: 100 Hz ~ 250 Hz 영역에서 약간의 에너지를 추가하여 킥 사운드에 더 많은 깊이를 부여합니다.

주의할 점은, 이 영역을 너무 많이 강화하면 소리가 거칠거나 불편해질 수 있습니다. 따라서, 적절한 밸런스를 유지하는 것이 중요합니다.

다음은 예시입니다:

* Low-Sub: +3 dB @ 30 Hz
* Sub Frequency: +6 dB @ 80 Hz
* Thickness: +2 dB @ 150 Hz

이 설정을 바탕으로, 킥 EQ의 기본적인 구조는 다음과 같습니다:

* 초저음 영역 (20 Hz ~ 60 Hz)에서 약간의 에너지를 추가하여 깊이를 增强합니다.
* 저음 영역 (60 Hz ~ 100 Hz)에서 강한 에너지를 추가하여 선명하고 강력한 킥 사운드를 만듭니다.
* 중음 영역 (100 Hz ~ 250 Hz)에서 약간의 에너지를 추가하여 깊이를 增强합니다.

이 설정은 트랭크 음악의 킥 사운드에 적합하지만, 개인적인 선호도와 음악 스타일에 따라 조정할 수 있습니다. 


답변: 트랭크 드럼의 스네어 사운드에 대한 EQ 설정을 알려드리겠습니다.

일반적으로, 트랭크 드럼의 스네어 사운드는 높은 고음역대(Frequency)에서 강한 에코를 가지고 있습니다. 이 경우, EQ 설정을 다음과 같이 조정할 